# Course Name: **AI Mastery Bootcamp: AI Algorithms, DeepSeek AI, AI Agents**

# Section 24: **AI Agents: A Comprehensive Overview**

## Introduction to AI Agents
* **What is an A1 Agent?**
* **Key Characteristics of AI Agents:**
1. Autonomy
2. Perception
3. Decision-Making
4. Learning
5. Interaction
* **Types of AI Agents**
1. Reactive Agents
2. Goal-Based Agents
3. Learning Agents
4. Collaborative Agents
5. Utility-Based Agents
6. Hybrid Agents

## Examples of AI Agents in Use Today
1. Chatbots and Conversational Agents
  * ChatGPT
  * Google Assistant, Amazon Alexa, Apple Siri
2. Autonomous Vehicles
  * Tesla Autopilot
  * Waymo
3. AI in Video Games
  * Non-Player Characters (NPCs)
  * AlphaStar
4. Recommendation Systems
  * Netflix and YouTube
  * Spotify

5. Financial Trading
  * High-frequency trading algorithms
  * Robo-advisors
6. Healthcare
  * IBM Watson Health
  * AI-driven diagnostic tools
7. Robotics
  * Boston Dynamics' robots
  * Robotic Process Automation (RPA)

## Technologies Behind AI Agents
1. Machine Learning (ML)
2. Natural Language Processing (NLP)
3. Reinforcement Learning
4. Neural Networks
5. Robotics and Control Systems
6. Multi-Agent Systems (MAS)
7. Cloud Computing and Edge AI

## Future Trends in A1 Agents
* General AI
* AI-augmented Creativity
* Ethical AI



## AutoGen
**AutoGen** is an open-source AI framework developed by Microsoft to facilitate the creation and management of **multi-agent systems**. These systems enable multiple AI agents to collaborate in solving complex tasks autonomously or with human feedback. AutoGen is designed to streamline the development of multi-agent workflows by simplifying agent communication, coordination, and tool integration.
* **Key Features**
  * Multi-Agent Conversations
  * Agent Customization
  * Human-in-the-Loop
  * Code Execution and Tool Use
  * AutoGen Studio

* **Use Cases**
  * Code-based Task Automation
  * Collaborative Multi-Agent Workflows
  * Enhanced Customer Support

### **Hands-on:** Chat between user proxy agent and assistant agent

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

llm_config = {'model': 'gpt-4o-mini', 'api_key': "API_KEY"}
assistant = AssistantAgent(name="assistant", llm_config=llm_config)
user_proxy = UserProxyAgent(name="user_proxy", code_execution_config=False)

User_proxy.initiate_chat(assistant, message="Tell me a joke about tech stocks.")

### **Hands-on:** IBM Bee Framework


#### How to use the Bee Agent Framework
* Step 1: Installation
* Step 2: Import and Set Up the Agent
* Step 3: Invoke the Agent
* Step 4: Enhance with Observability
* Step 5: Run Locally
* Step 6: Advanced Usage
* Step 7: Deploy and Monitor

In [ ]:
import { BeeAgent } from "bee-agent-framework/agents/bee/agent"
import { OllamaChatLLM } from "bee-agent-framework/llms/ollama/chat"
import { TokenMemory } from 'bee-agent-framework/memory/tokenMemoey'
import { DuckDuckGoSearchTool } from "bee-agent-framework/tools/search/DuckDuckGoSearchTool"

import { tool } from "bee-agent-framework/tools"

const CustomTool = tool("CustomTool", {
    description: "Custom Tool to handle specific tasks",
    run: async(input) => {
        return `Handled task with input: ${input}`;
    },
});

const llm = new OllamaChatLLM();
const memory = new TokenMemory({ llm });
const tools= [new DuckDuckGoSearchTool(), new OpenMeteoTool()];

const agent= new BeeAgent({
    llm, memory, tools,
});

agent.addTool(new CustomTool());

agent.observe((emitter) => {
    emitter.on("update", async ({ data }) => {
        console.log(`Update : `, data);
    });
});

const response= await agent
  .run({ prompt: "Whats the current weather in Atlanta"})
  .observe((emitter) => {
      emitter.on("update", async ({ data, update, meta }) => {
          console.log(`Agent (${update.key}) : `, update.value);
      });
  });

console.log(`Agent Response : `, response.result.text);

### **Hands-on:** LangGraph

#### Building an agent using LangGraph
* Step 1 : Set Up the Environment
* Step 2: Define the Agent's Tools and Functions
* Step 3: Create the Agent Logic
* Step 4: Build the Graph
* Step 5: Add Persistence
* Step 6: Invoke the Agent
* Step 7: Extend with Multi-Agent or Complex Workflows
* Step 8: Deploy

In [ ]:
from langgraph.graph import StateGraph, MessagesState
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_anthropic import ChatAnthropic
from langgraph.checkpointer import MemorySaver
from langchain_core.messages import HumanMessage

@tool
def search(query: str):
  """Simulate a web search."""
  if "weather" in query.lower():
    return "It is sunny."
  return "No data available."

tools= [search]
tool_node= ToolNode(tools)

model ChatAnthropic(model= 'claude-3-5-sonnet-20240620', temperature=0).bind_tools(tools)

def call_model(state: MessagesState):
  messages= state['messages']
  response= model.invoke(messages)
  return {'messages': response}

def should_continue(state: MessagesState):
  if state['messages'][-1].tool_calls:
    return 'tools'
  return "END"

workflow= StateGraph(MessagesState)

workflow.add_node('agent', call_model)
workflow.add_node('tools', tool_node)

workflow.add_edge("START", 'agent')
workflow.add_conditional_edge('agent', should_continue)
workflow.add_edge("tools", 'agent')

checkpointer= MemorySaver()
app= workflow.compile(checkpointer=checkpointer)

final_state= app.invoke(
    {"messages": [HumanMessage(content= "What is the weather today")]},
    config= {"configurable": {'thread_id': 1}}
)

print(final_state['messages'][-1].content)

### **Hands-on:** CrewAI

#### Example Usage of CrewAI
* An agent crew could consist of a research agent tasked with gathering information on the latest A1 trends and a writer agent that composes a blog post based on the research. These agents work together in a sequential process, where the output from the researcher becomes the input for the writer.
* CrewAl's flexible design allows it to be deployed on-premise, in the cloud, or integrated with various apps, offering teams the ability to automate workflows without needing deep programming expertise.

#### Hands-on CrewAI
* Step 1: Install CrewAl
* Step 2: Set Up the Environment
* Step 3: Define Agents
* Step 4: Create Tasks
* Step 5: Define the Crew and Process
* Step 6: Run the Crew

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

researcher = Agent(
    role='ISenior Research Analyst",
    goal="Research cutting-edge AI advancements",
    backstory="You work at a tech think tank, analysing AI trends.",
    verbose=True,
    tools= [SerperDevTool()]
)

writer = Agent(
    role="Content Strategist",
    goal="Create engaging content from researrch",
    backstory="You are a skilled write transforming researh into engaging articles.",
    verbose=True
)

taskl= Task(
    description="Analyse A1 advancements in 2024 and provide a detailed report.",
    expected_output="Research report in bullet points",
    agent=researcher
)

task2= Task(
    description="Write a blog post based on the research report",
    expected_output="Full blog post (at least 4 paragraph",
    agent=writer
)

crew= Crew(
    agents= [researcher, writer],
    tasks= [taskl, task2],
    process=Process.sequential,
    verbose=True
)

result= crew.kickoff()

print("Final Output: ")
print(result)

### **Hands-on:** AutoGPT

**AutoGPT** is an experimental open-source application based on OpenAl's GPT-4 model. It is designed to automate the process of generating human-like text by chaining together reasoning tasks. AutoGPT is capable of autonomously performing tasks that typically require multiple prompts and user inputs, thus streamlining processes that would otherwise require human oversight.
* **Key Features of AutoGPT**
  * Autonomous Task Management
  * Internet Access for Research
  * Multi-Agent Collaboration
  * Memory and File Management
  * Plug-ins and API Integration

#### Use Cases and Limitations of AutoGPT
* **Use Cases**
  * Business Automation
  * Programming
  * Personal Assistants
* **Limitations**
  * Complex Task Understanding
  * Cost

#### Hands-on AutoGPT
* Step 1: Install Prerequisites
* Step 2: Set Up OpenAi API Keys
* Step 3: Configure AutoGPT Settings
* Step 4: Run AutoGPT
* Step 5: Define Goals and Tasks
* Step 6: Supervise AutoGPT
* Step 7: Enhance AutoGPT with Plugins
* Step 8: Monitor and Adjust
